## Paso 1 — Carga de datos y exploración mínima

En este primer paso se carga el conjunto de datos *Khan* y se realiza una revisión inicial para asegurar su integridad antes del análisis. Se detecta automáticamente la columna de etiquetas, buscando nombres comunes como `class` o `label`, y si no se encuentra se utiliza la última columna disponible. Posteriormente, se muestran las dimensiones del conjunto, el conteo de observaciones por clase y la cantidad de valores faltantes tanto totales como por columna, si existieran. Finalmente, se calculan las diferencias de medias entre las clases 2 y 4 para todos los genes, destacando los diez con mayor valor absoluto, con el fin de observar indicios tempranos de separación entre grupos.


In [1]:
# ===== Imports base (reusados en los pasos siguientes) =====
import pandas as pd, numpy as np
from scipy.stats import ttest_ind, f_oneway
from statsmodels.stats.multitest import multipletests
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
np.random.seed(42)

# ===== Carga y preparación mínima =====
df = pd.read_csv("A3.1 Khan.csv")
y = df["y"].astype(int)          # columna de etiqueta fija: 'y'
X = df.drop(columns=["y"])       # resto son genes

print("Dimensiones:", df.shape)
print("Conteo por clase:")
print(y.value_counts().sort_index())

# ===== Huecos =====
print("\nHuecos totales en el dataset:")
print(df.isnull().sum().sum())

grupo2 = df[df["y"] == 2]
grupo4 = df[df["y"] == 4]

# Diferencia de medias 2 - 4 por gen (mantengo vector completo para usos posteriores)
mean_diff = (grupo2.mean(numeric_only=True) - grupo4.mean(numeric_only=True))

# Top-10 por valor absoluto de la diferencia (presentación compacta)
diferencia_abs = mean_diff.abs().sort_values(ascending=False)
top10 = diferencia_abs.head(10)

print("\nTop-10 |Δ media| (clase 2 vs 4):")
display(top10.to_frame("abs(mean_2 - mean_4)"))

# Máscaras útiles para pasos posteriores
mask2, mask4 = (y == 2), (y == 4)


Dimensiones: (83, 2309)
Conteo por clase:
y
1    11
2    29
3    18
4    25
Name: count, dtype: int64

Huecos totales en el dataset:
0

Top-10 |Δ media| (clase 2 vs 4):


,abs(mean_2 - mean_4)
X187,3.323151
X509,2.906537
X2046,2.424515
X2050,2.401783
X129,2.165185
X1645,2.065460
X1319,2.045941
X1955,2.037340
X1003,2.011337
y,2.000000


## Paso 2 — Pruebas de hipótesis por gen (2 vs 4) y ajustes por comparaciones múltiples

Comparamos, para cada gen, la expresión entre las clases 2 y 4 usando una **t de Welch** (`equal_var=False`) y después corregimos los p-values por comparaciones múltiples con **Bonferroni**, **Holm** (control FWER) y **Benjamini–Hochberg** (control FDR), fijando **α = 0.05**. Reportaremos cuántos genes resultan significativos con cada método, mostraremos un **Top-10** ordenado por p crudo incluyendo sus p ajustados y conservaremos la **tabla completa** para reutilizarla en pasos posteriores.


In [2]:
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests

# Columnas de genes (todo excepto la etiqueta 'y')
genes = [c for c in df.columns if c != "y"]

# Welch t-test por gen (2 vs 4)
t_stats, p_values = [], []
for gen in genes:
    t, p = ttest_ind(grupo2[gen], grupo4[gen], equal_var=False)
    t_stats.append(t)
    p_values.append(p)

# Tabla completa con p crudo y p ajustados
resultados = pd.DataFrame({
    "gen": genes,
    "t_stat": t_stats,
    "p_raw": p_values
})
resultados["p_bonf"] = multipletests(resultados["p_raw"], method="bonferroni")[1]
resultados["p_holm"] = multipletests(resultados["p_raw"], method="holm")[1]
resultados["p_fdr_bh"] = multipletests(resultados["p_raw"], method="fdr_bh")[1]

# Banderas de significancia (α=0.05)
resultados["sig_bonf"] = resultados["p_bonf"] < 0.05
resultados["sig_holm"] = resultados["p_holm"] < 0.05
resultados["sig_fdr_bh"] = resultados["p_fdr_bh"] < 0.05

# Conteos
print("Genes significativos (α = 0.05)")
print("Bonferroni:", resultados["sig_bonf"].sum())
print("Holm     :", resultados["sig_holm"].sum())
print("FDR (BH) :", resultados["sig_fdr_bh"].sum())

# TOP por cada criterio (ordenados por su p correspondiente)
print("\nTop-10 por p BONFERRONI:")
display(resultados.sort_values("p_bonf")[["gen","p_bonf","p_raw","p_holm","p_fdr_bh"]].head(10))

print("\nTop-10 por p HOLM:")
display(resultados.sort_values("p_holm")[["gen","p_holm","p_raw","p_bonf","p_fdr_bh"]].head(10))

print("\nTop-10 por p FDR (BH):")
display(resultados.sort_values("p_fdr_bh")[["gen","p_fdr_bh","p_raw","p_bonf","p_holm"]].head(10))


Genes significativos (α = 0.05)
Bonferroni: 72
Holm     : 72
FDR (BH) : 296

Top-10 por p BONFERRONI:


,gen,p_bonf,p_raw,p_holm,p_fdr_bh
1002,X1003,1.153698e-13,4.998692e-17,1.153698e-13,1.153698e-13
186,X187,8.578576e-13,3.716887e-16,8.574859e-13,4.289288e-13
2049,X2050,9.427801e-12,4.084836e-15,9.419631e-12,3.062213e-12
1954,X1955,1.224885e-11,5.307128e-15,1.223293e-11,3.062213e-12
1644,X1645,1.907075e-11,8.262889e-15,1.903770e-11,3.814150e-12
245,X246,3.548567e-11,1.537507e-14,3.540879e-11,5.833619e-12
2045,X2046,4.083533e-11,1.769295e-14,4.072917e-11,5.833619e-12
508,X509,1.743776e-10,7.555354e-14,1.738487e-10,2.179720e-11
1953,X1954,2.193537e-10,9.504059e-14,2.185934e-10,2.437263e-11
1388,X1389,1.474328e-09,6.387904e-13,1.468579e-09,1.474328e-10



Top-10 por p HOLM:


,gen,p_holm,p_raw,p_bonf,p_fdr_bh
1002,X1003,1.153698e-13,4.998692e-17,1.153698e-13,1.153698e-13
186,X187,8.574859e-13,3.716887e-16,8.578576e-13,4.289288e-13
2049,X2050,9.419631e-12,4.084836e-15,9.427801e-12,3.062213e-12
1954,X1955,1.223293e-11,5.307128e-15,1.224885e-11,3.062213e-12
1644,X1645,1.903770e-11,8.262889e-15,1.907075e-11,3.814150e-12
245,X246,3.540879e-11,1.537507e-14,3.548567e-11,5.833619e-12
2045,X2046,4.072917e-11,1.769295e-14,4.083533e-11,5.833619e-12
508,X509,1.738487e-10,7.555354e-14,1.743776e-10,2.179720e-11
1953,X1954,2.185934e-10,9.504059e-14,2.193537e-10,2.437263e-11
1388,X1389,1.468579e-09,6.387904e-13,1.474328e-09,1.474328e-10



Top-10 por p FDR (BH):


,gen,p_fdr_bh,p_raw,p_bonf,p_holm
1002,X1003,1.153698e-13,4.998692e-17,1.153698e-13,1.153698e-13
186,X187,4.289288e-13,3.716887e-16,8.578576e-13,8.574859e-13
1954,X1955,3.062213e-12,5.307128e-15,1.224885e-11,1.223293e-11
2049,X2050,3.062213e-12,4.084836e-15,9.427801e-12,9.419631e-12
1644,X1645,3.814150e-12,8.262889e-15,1.907075e-11,1.903770e-11
245,X246,5.833619e-12,1.537507e-14,3.548567e-11,3.540879e-11
2045,X2046,5.833619e-12,1.769295e-14,4.083533e-11,4.072917e-11
508,X509,2.179720e-11,7.555354e-14,1.743776e-10,1.738487e-10
1953,X1954,2.437263e-11,9.504059e-14,2.193537e-10,2.185934e-10
1388,X1389,1.474328e-10,6.387904e-13,1.474328e-09,1.468579e-09


Tras aplicar la prueba t de Welch a cada gen para comparar la expresión entre las clases 2 y 4, se identificó un conjunto considerable de genes con diferencias estadísticamente significativas.
Los métodos Bonferroni y Holm, que son más conservadores al controlar el error familiar, detectaron 72 genes significativos cada uno. En cambio, el método Benjamini–Hochberg (FDR-BH), al permitir un pequeño margen de falsos positivos, identificó 296 genes como significativos, lo que sugiere una mayor sensibilidad a cambios reales en la expresión.

En todos los métodos, los genes X1003, X187 y X2050 se mantienen entre los más significativos, coincidiendo con aquellos que ya mostraban las mayores diferencias de medias en el Paso 1.
Esto refuerza su relevancia biológica potencial y respalda la consistencia entre los análisis exploratorios y las pruebas inferenciales.

En resumen, los resultados confirman la presencia de múltiples genes con comportamiento diferencial claro entre las clases 2 y 4, consolidando la evidencia estadística de que estos grupos presentan perfiles de expresión génica distintivos.

## Paso 3 — ANOVA multiclase (4 clases) con corrección FDR

- Prueba: **ANOVA de una vía** (`scipy.stats.f_oneway`) por gen.
- Corrección por comparaciones múltiples: **FDR Benjamini–Hochberg** (α = 0.05).
- Reportes a mostrar:
  1) Número total de genes significativos (FDR ≤ 0.05).
  2) Tabla **Top-10** por *p-value* crudo con su *p-value* ajustado (FDR).
  3) Guardar la tabla completa para uso posterior.


In [3]:
from scipy.stats import f_oneway
from statsmodels.stats.multitest import multipletests

genes = [c for c in df.columns if c != "y"]

# ANOVA de una vía (4 clases)
p_values_anova = []
for gen in genes:
    grupos = [df[df["y"] == c][gen] for c in sorted(df["y"].unique())]
    _, p = f_oneway(*grupos)
    p_values_anova.append(p)

# Tabla de resultados
anova_res = pd.DataFrame({
    "gen": genes,
    "p_raw": p_values_anova
})

# Corrección FDR-BH
anova_res["p_fdr_bh"] = multipletests(anova_res["p_raw"], method="fdr_bh")[1]
anova_res["sig_fdr_bh"] = anova_res["p_fdr_bh"] < 0.05

# Conteo de genes significativos
n_sig = anova_res["sig_fdr_bh"].sum()
print(f"Genes significativos por ANOVA (FDR BH, α=0.05): {n_sig}")

# Top-10 por p-value crudo
print("\nTop-10 genes por p-value crudo (con p FDR BH):")
display(anova_res.sort_values("p_raw").head(10)[["gen", "p_raw", "p_fdr_bh"]])

# Guardar tabla completa
res_anova = anova_res.sort_values("p_raw").reset_index(drop=True)


Genes significativos por ANOVA (FDR BH, α=0.05): 1162

Top-10 genes por p-value crudo (con p FDR BH):


,gen,p_raw,p_fdr_bh
1954,X1955,1.459035e-24,2.045755e-21
1388,X1389,1.772751e-24,2.045755e-21
1002,X1003,1.618988e-23,1.245542e-20
2049,X2050,4.733702e-22,2.731346e-19
245,X246,6.633722e-22,3.062126e-19
741,X742,2.195548e-21,8.445542e-19
0,X1,3.839240e-20,1.265852e-17
2161,X2162,1.035143e-19,2.986387e-17
1953,X1954,2.182635e-19,5.597246e-17
1644,X1645,2.988392e-19,6.751740e-17


El análisis ANOVA entre las cuatro clases identificó 1,162 genes significativos después de aplicar la corrección FDR Benjamini–Hochberg (α = 0.05).
Esto muestra que muchos genes presentan diferencias reales en sus niveles de expresión entre los distintos grupos.

Los genes X1955, X1389 y X1003 destacan nuevamente entre los más significativos, coincidiendo con los resultados obtenidos en los pasos anteriores.
Esto sugiere que estos genes podrían estar estrechamente relacionados con las características que diferencian a las clases del conjunto de datos.

## Paso 4 — Modelado con SVM (kernels lineal, polinomial grado 3 y RBF)

- **Selección de variables:** usar los genes más asociados al comportamiento multiclase según ANOVA (Paso 3). Se seleccionan los **k** genes con menor *p* ajustado (FDR BH).
- **División:** 75% entrenamiento / 25% prueba estratificado.
- **Preprocesamiento:** `StandardScaler` dentro de un `Pipeline` para cada modelo.


In [12]:
# === Paso 4: Entrenamiento SVM con 3 kernels y comparación de desempeño ===
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

# ----- 1) Selección de variables desde ANOVA (Paso 3) -----
# k ajustable: si tu máquina va justa, baja a 50; si va bien, 100–200 suele funcionar
k = 100
assert 'res_anova' in globals(), "No encuentro la tabla de ANOVA (Paso 3). Ejecuta el Paso 3 antes del 4."
topk_genes = res_anova.sort_values("p_fdr_bh").head(k)["gen"].tolist()
Xk = X[topk_genes].copy()

# ----- 2) Split train/test estratificado -----
X_train, X_test, y_train, y_test = train_test_split(
    Xk, y, test_size=0.25, stratify=y, random_state=42
)

# ----- 3) Definir modelos -----
models = {
    "linear": Pipeline([("scaler", StandardScaler()),
                        ("clf", SVC(kernel="linear", random_state=42))]),
    "poly3":  Pipeline([("scaler", StandardScaler()),
                        ("clf", SVC(kernel="poly", degree=3, coef0=0, random_state=42))]),
    "rbf":    Pipeline([("scaler", StandardScaler()),
                        ("clf", SVC(kernel="rbf", gamma="scale", random_state=42))]),
}

# ----- 4) Entrenar y evaluar -----
scores = {}
cms = {}
reports = {}

for name, pipe in models.items():
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    acc  = accuracy_score(y_test, y_pred)
    f1m  = f1_score(y_test, y_pred, average="macro")
    cm   = confusion_matrix(y_test, y_pred, labels=sorted(y.unique()))
    rpt  = classification_report(y_test, y_pred, digits=3)
    scores[name]  = {"accuracy": acc, "macroF1": f1m}
    cms[name]     = cm
    reports[name] = rpt


## Paso 5: Conclusiones 
- **Métricas de comparación:** `accuracy` y **macro-F1** (promedia por clase). Además, se muestra la **matriz de confusión**.
- **Criterio de mejor modelo:** mayor **macro-F1** en el conjunto de prueba.


In [13]:
# ----- 5) Mostrar resumen comparativo -----
print("Resumen de métricas en TEST (k =", k, "genes):")
for name, sc in scores.items():
    print(f"- {name:6s} | accuracy={sc['accuracy']:.3f} | macroF1={sc['macroF1']:.3f}")

# Mejor por macro-F1
best_name = max(scores, key=lambda n: scores[n]["macroF1"])
print("\nMejor kernel por macro-F1:", best_name)

# Matrices de confusión y reportes (útiles para la redacción)
for name in models.keys():
    print(f"\n[{name}] Matriz de confusión (clases ordenadas):")
    print(cms[name])
    print(f"\n[{name}] Classification report:")
    print(reports[name])


Resumen de métricas en TEST (k = 100 genes):
- linear | accuracy=1.000 | macroF1=1.000
- poly3  | accuracy=0.905 | macroF1=0.906
- rbf    | accuracy=1.000 | macroF1=1.000

Mejor kernel por macro-F1: linear

[linear] Matriz de confusión (clases ordenadas):
[[3 0 0 0]
 [0 7 0 0]
 [0 0 5 0]
 [0 0 0 6]]

[linear] Classification report:
              precision    recall  f1-score   support

           1      1.000     1.000     1.000         3
           2      1.000     1.000     1.000         7
           3      1.000     1.000     1.000         5
           4      1.000     1.000     1.000         6

    accuracy                          1.000        21
   macro avg      1.000     1.000     1.000        21
weighted avg      1.000     1.000     1.000        21


[poly3] Matriz de confusión (clases ordenadas):
[[3 0 0 0]
 [0 7 0 0]
 [0 2 3 0]
 [0 0 0 6]]

[poly3] Classification report:
              precision    recall  f1-score   support

           1      1.000     1.000     1.000       


- El modelo **lineal** clasificó correctamente todos los casos de prueba (100 % de precisión y recall por clase).  
- El **RBF** también alcanzó resultados idénticos, confirmando que el espacio de características ya es linealmente separable.  
- El kernel **polinomial** mostró un leve descenso (accuracy = 0.905) debido a sobreajuste o exceso de complejidad.  

**Conclusión general:** el conjunto *Khan* presenta un gran número de genes con diferencias de expresión estadísticamente significativas entre tipos tumorales.  
En términos predictivos, un **SVM lineal** con una selección concisa de genes obtenidos por inferencia multiclase ofrece un desempeño **óptimo y parsimonioso**.

